In [1]:
%%capture

!apt-get -y install protobuf-compiler python-pil python-lxml
!pip install pystream-protobuf

In [2]:
s = '''syntax = "proto2";

package music_io;

message Item {
  required Matrix features = 1;
  optional string item_class = 2;
  optional string item_review = 3;
  repeated double item_targets = 4;
  repeated string token_class = 5;
}

message Matrix {
  required int32 feature_size = 1; 
  repeated double flat_matrix = 2;
}
'''

# We dont consider Token regression on purpose in this case

with open("source/io/item.proto", "w") as f:
    f.write(s)

In [3]:
!protoc --python_out=. source/io/item.proto

In [1]:
import numpy as np
from source.io.pb_item import pb_Item

features = np.random.rand(10,20)

x = pb_Item(features,
            seq_class="rock",
            seq_targets=[1.5, 3.5],
            token_class=["ciao", "pippo"],
            review="This is an example review")

In [6]:
import stream

with stream.open("source/io/test.bin", "ab") as ostream:
    ostream.write(x.get_pb_obj())

In [7]:
with stream.open("source/io/test.bin", "rb") as istream:
    for data in istream:
        x = pb_Item()
        x.load_from_string(data)
        
        print(x)